In [ ]:
import sys
import AE_models

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, KFold
import pandas as pd
import numpy as np
import keras 
from keras import layers
import scipy.io as sio
import pandas as pd 

import matplotlib.pyplot as plt
import time

## 1.Functions

In [ ]:
def value_range_analysis(X):
    if sum(sum(((np.round(X,4) <= 1)) & (X >= 0))) == X.shape[0] * X.shape[1]:
        print("Range value: [0,1]")
    else:
        print("ERROR!!")

In [ ]:
def plotLossesLayers(h_layers, dicc, index_data, data):
    plot_layers = []
    for i in range(len(h_layers)):
        plot_layers.append(h_layers[i][1])

    keys = list(dicc.keys())
    print(len(keys))
    
    filas = int(np.ceil(len(keys)/2))
    columnas = int(np.sqrt(np.round(len(keys)/2)))
    print(filas)
    print(columnas)
    f, axis = plt.subplots(filas, columnas, figsize=(20,20))

    index_keys = 0
    for i in range(filas):
        for j in range(columnas):
            # error in validation
            axis[i,j].plot(plot_layers, dicc[keys[index_keys]][0])
            # error in train
            axis[i,j].plot(plot_layers, dicc[keys[index_keys]][1])
            axis[i,j].legend(["val_loss", "loss"])
            axis[i,j].set_title(keys[i])
            index_keys += 1

    plt.close(f)


## 2. FE

In [ ]:
folders = ["S1", "S2", "S3", "S4", "S5"]
normalizar = True
plotAndSaveImg_train = False
typeData = "FE"

for index_data in range(len(folders)):
    print("================================================================>" + folders[index_data] + "<=======================================================")
    X_train_original, X_test_original, y_train_original, y_test_original = AE_models.loadData(index_data, normalizar, typeData, folders)
    value_range_analysis(X_train_original)
    
    autoencodertype = {'DAE': False, 'AE': True}
    
    epochs = 5000
    initial_learning_rate = [1e-4, 1e-3, 1e-2, 1e-1]
    dropout=[0, .05, .1]
    batch_size = [32]
    h_layers_2 = [[X_train_original.shape[1], 140],
                [X_train_original.shape[1], 135], 
                [X_train_original.shape[1], 130],
                [X_train_original.shape[1], 125],
                [X_train_original.shape[1], 120],
                [X_train_original.shape[1], 115],
                [X_train_original.shape[1], 110],
                [X_train_original.shape[1], 100]]


    hyperparameters = {'epochs': epochs,
                       'initial_learning_rate':initial_learning_rate,
                       'kfold':5,
                       "mindelta": 0}

    y_pre_train = y_train_original
    y_test = y_test_original
    X_pre_train = X_train_original
    X_test = X_test_original

    semillas = [9,18,35, 52, 75]

    kf = KFold(n_splits=hyperparameters["kfold"], shuffle=True, random_state=semillas[index_data])
    kf.get_n_splits(X_pre_train)

    bestHyperparameters_2 = {'h_layers': -1, 'batch_size': -1, 'initial_learning_rate': -1 }

    bestMetricDev = np.inf

    dicc_2 = {}
    for init in range(len(initial_learning_rate)):
        for bs in range(len(batch_size)):
            for d in range(len(dropout)):
                plot_val_loss_mean = []
                plot_loss_mean = []
                for l in range(len(h_layers_2)):
                    hyperparameters_aux = {'epochs': hyperparameters['epochs'],
                                       'initial_learning_rate': hyperparameters['initial_learning_rate'][init],
                                       "mindelta": hyperparameters["mindelta"],
                                       'batch_size': batch_size[bs],
                                       'dropout': dropout[d],
                                       'h_layers': h_layers_2[l], 'verbose': 0}

                    v_metric_dev = []
                    val_loss = []
                    loss = []
                    for train_index, val_index in kf.split(X_pre_train):
                        X_train, X_val = X_pre_train[train_index], X_pre_train[val_index]
                        # Reset keras
                        AE_models.reset_keras()
                        # Train the network and test it
                        model, hist, early = AE_models.runNetwork(X_train, X_val,
                                                                  hyperparameters_aux,
                                                                  autoencodertype)


                        loss.append(np.min(hist.history["loss"]))
                        val_loss.append(np.min(hist.history["val_loss"]))

                    plot_val_loss_mean.append(np.mean(val_loss))
                    plot_loss_mean.append(np.mean(loss))

                    metric_dev = np.mean(val_loss)
                    print("%%%%%%%%%bestHyperparameters%%%%%%%%%")
                    print(bestHyperparameters_2)

                    if metric_dev < bestMetricDev:
                        print("\tChange the best ", bestMetricDev, " by metric dev: ", metric_dev)
                        bestMetricDev = metric_dev
                        bestHyperparameters_2['h_layers'] = l
                        bestHyperparameters_2['batch_size'] = bs
                        bestHyperparameters_2['initial_learning_rate'] = init
                        bestHyperparameters_2['dropout'] = d
                
                dicc_2["bs:" + str(batch_size[bs]) + "-lr:" + str(initial_learning_rate[init]) + "-drop:" + str(dropout[d])] = [plot_val_loss_mean, plot_loss_mean]

    if plotAndSaveImg_train:
        plotLossesLayers(h_layers_2, dicc_2, index_data, typeData)

    print("Layers selected:", h_layers_2[bestHyperparameters_2["h_layers"]])
    print("dropout selected:", dropout[bestHyperparameters_2["dropout"]])
    print("batch_size selected:", batch_size[bestHyperparameters_2["batch_size"]])
    print("initial_learning_rate selected:", initial_learning_rate[bestHyperparameters_2["initial_learning_rate"]])

    print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%================================================================> BEST MODEL SELECT WITH DIFFERENTS HYPERPARAMETERS <=======================================================%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")

    hyperparameters = {'epochs': hyperparameters['epochs'],
                   'initial_learning_rate': initial_learning_rate[bestHyperparameters_2["initial_learning_rate"]],
                   "mindelta": hyperparameters["mindelta"],
                   'batch_size': batch_size[bestHyperparameters_2["batch_size"]],
                   'h_layers': h_layers_2[bestHyperparameters_2["h_layers"]],
                   'dropout': dropout[bestHyperparameters_2["dropout"]],
                   'verbose': 1}
    
    X_train_aux, X_val_aux = train_test_split(X_pre_train,
                                              test_size=0.3,
                                              random_state=semillas[index_data])
    
    autoencoder, hist, early = AE_models.runNetwork(X_train_aux, X_val_aux,
                                                    hyperparameters,
                                                    autoencodertype)

    encoder_2 = keras.Model(autoencoder.input, autoencoder.get_layer('Latent_layer').output)
    # Load the data
    X_train = pd.read_csv('../../data_generated_by_statistics/' + typeData + '/' + folders[index_data] + '/X_train.csv')
    X_test = pd.read_csv('../../data_generated_by_statistics/' + typeData + '/' + folders[index_data] + '/X_test.csv')
    
    X_train, X_test = AE_models.normData_minmax(X_train, X_test)

    encoder_2 = keras.Model(autoencoder.input, autoencoder.get_layer('Latent_layer').output)
    # Encode the data
    X_train_encode_2 = encoder_2.predict(X_train)
    X_test_encode_2 = encoder_2.predict(X_test)

    pd.DataFrame(X_train_encode_2).to_csv("../../1_Clasifications_models/data_reduced/"+ typeData +"/AE/X_train_Norm_AE_" + folders[index_data] + ".csv", index=False)
    pd.DataFrame(X_test_encode_2).to_csv("../../1_Clasifications_models/data_reduced/"+ typeData +"/AE/X_test_Norm_AE_" + folders[index_data] + ".csv", index=False)

## 3. FE_kernel

In [ ]:
folders = ["S1", "S2", "S3", "S4", "S5"]
normalizar = True
plotAndSaveImg_train = False
typeData = "FE_kernel"

for index_data in range(len(folders)):
    print("================================================================>" + folders[index_data] + "<=======================================================")
    X_train_original, X_test_original, y_train_original, y_test_original = AE_models.loadData(index_data, normalizar, typeData, folders)
    value_range_analysis(X_train_original)
    
    autoencodertype = {'DAE': False, 'AE': True}
    
    epochs = 5000
    initial_learning_rate = [1e-4, 1e-3, 1e-2, 1e-1]
    dropout=[0, .05, .1]
    batch_size = [32]
    h_layers_3 = [[X_train_original.shape[0], 450],
                [X_train_original.shape[0], 425],
                [X_train_original.shape[0], 400],
                [X_train_original.shape[0], 375],
                [X_train_original.shape[0],  350],
                [X_train_original.shape[0],  325],
                [X_train_original.shape[0],  300],
                [X_train_original.shape[0], 275],
                [X_train_original.shape[0], 250],
                [X_train_original.shape[0], 225],
                [X_train_original.shape[0], 200]]

    hyperparameters = {'epochs': epochs,
                       'initial_learning_rate':initial_learning_rate,
                       'kfold':5,
                       "mindelta": 0.0001}

    y_pre_train = y_train_original
    y_test = y_test_original
    X_pre_train = X_train_original
    X_test = X_test_original

    semillas = [9,18,35, 52, 75]

    kf = KFold(n_splits=hyperparameters["kfold"], shuffle=True, random_state=semillas[index_data])
    kf.get_n_splits(X_pre_train)

    bestHyperparameters_3 = {'h_layers': -1, 'batch_size': -1, 'initial_learning_rate': -1 }

    bestMetricDev = np.inf

    dicc_3 = {}
    for init in range(len(initial_learning_rate)):
        for bs in range(len(batch_size)):
            for d in range(len(dropout)):
                plot_val_loss_mean = []
                plot_loss_mean = []
                for l in range(len(h_layers_3)):
                    hyperparameters_aux = {'epochs': hyperparameters['epochs'],
                                       'initial_learning_rate': hyperparameters['initial_learning_rate'][init],
                                       "mindelta": hyperparameters["mindelta"],
                                       'batch_size': batch_size[bs],
                                       'dropout': dropout[d],
                                       'h_layers': h_layers_3[l], 'verbose': 0}

                    v_metric_dev = []
                    val_loss = []
                    loss = []
                    for train_index, val_index in kf.split(X_pre_train):
                        X_train, X_val = X_pre_train[train_index], X_pre_train[val_index]
                        # Reset keras
                        AE_models.reset_keras()
                        # Train the network and test it
                        model, hist, early = AE_models.runNetwork(X_train, X_val,
                                                                  hyperparameters_aux,
                                                                  autoencodertype)


                        loss.append(np.min(hist.history["loss"]))
                        val_loss.append(np.min(hist.history["val_loss"]))

                    plot_val_loss_mean.append(np.mean(val_loss))
                    plot_loss_mean.append(np.mean(loss))

                    metric_dev = np.mean(val_loss)

                    if metric_dev < bestMetricDev:
                        print("\tChange the best ", bestMetricDev, "by metric dev: ", metric_dev)
                        bestMetricDev = metric_dev
                        bestHyperparameters_3['h_layers'] = l
                        bestHyperparameters_3['batch_size'] = bs
                        bestHyperparameters_3['initial_learning_rate'] = init
                        bestHyperparameters_3['dropout'] = d
                        print("%%%%%%%%%bestHyperparameters%%%%%%%%%")
                        print(bestHyperparameters_3)

                dicc_3["bs:" + str(batch_size[bs]) + "-lr:" + str(initial_learning_rate[init]) + "-drop:" + str(dropout[d])] = [plot_val_loss_mean, plot_loss_mean]

    if plotAndSaveImg_train:
        plotLossesLayers(h_layers_3, dicc_3, index_data, typeData)

    print("Layers selected:", h_layers_3[bestHyperparameters_3["h_layers"]])
    print("dropout selected:", dropout[bestHyperparameters_3["dropout"]])
    print("batch_size selected:", batch_size[bestHyperparameters_3["batch_size"]])
    print("initial_learning_rate selected:", initial_learning_rate[bestHyperparameters_3["initial_learning_rate"]])

    print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%================================================================> BEST MODEL SELECT WITH DIFFERENTS HYPERPARAMETERS <=======================================================%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")

    hyperparameters = {'epochs': hyperparameters['epochs'],
                   'initial_learning_rate': initial_learning_rate[bestHyperparameters_3["initial_learning_rate"]],
                   "mindelta": hyperparameters["mindelta"],
                   'batch_size': batch_size[bestHyperparameters_3["batch_size"]],
                   'h_layers': h_layers_3[bestHyperparameters_3["h_layers"]],
                   'dropout': dropout[bestHyperparameters_3["dropout"]],
                   'verbose': 1}
    
    X_train_aux, X_val_aux = train_test_split(X_pre_train,
                                              test_size=0.3,
                                              random_state=semillas[index_data])
    
    autoencoder, hist, early = AE_models.runNetwork(X_train_aux, X_val_aux,
                                                    hyperparameters,
                                                    autoencodertype)

    encoder_3 = keras.Model(autoencoder.input, autoencoder.get_layer('Latent_layer').output)
    # Load the data
    X_train = pd.read_csv('../../data_generated_by_statistics/' + typeData + '/' + folders[index_data] + '/X_train.csv')
    X_test = pd.read_csv('../../data_generated_by_statistics/' + typeData + '/' + folders[index_data] + '/X_test.csv')

    X_train, X_test = AE_models.normData_minmax(X_train, X_test)

    encoder_3 = keras.Model(autoencoder.input, autoencoder.get_layer('Latent_layer').output)
    # Encode the data
    X_train_encode_3 = encoder_3.predict(X_train)
    X_test_encode_3 = encoder_3.predict(X_test)

    pd.DataFrame(X_train_encode_3).to_csv("../../1_Clasifications_models/data_reduced/"+ typeData +"/AE/X_train_Norm_AE_" + folders[index_data] + ".csv", index=False)
    pd.DataFrame(X_test_encode_3).to_csv("../../1_Clasifications_models/data_reduced/"+ typeData +"/AE/X_test_Norm_AE_" + folders[index_data] + ".csv", index=False)